In [ ]:
!pip install -q kaggle

In [ ]:
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

In [7]:
#Data Extraction
#unzipping the file
import zipfile
# Unzip using Python
with zipfile.ZipFile("orders.csv.zip", "r") as zip_ref:
    zip_ref.extractall(".")  # Extract to the current directory
print("Unzipping complete!")

Unzipping complete!


In [ ]:
!pip install pandas

In [ ]:
#DATA CLEANING 
#read file
import pandas as pd
df = pd.read_csv("orders.csv",na_values=['Not Available','unknown'])   # changing null value into nan
#df['Ship Mode'].unique()  #to find unique values in ship mode 
df.columns = df.columns.str.lower()  #changed to lower case
df.columns = df.columns.str.replace(" ", "_")  #replace space with _
df["discount"]=df["list_price"] * (df["discount_percent"] / 100)    # calculating discount and added discount column
df["sale_price"]=df["list_price"]-df["discount"]  # sell price
df["profit"]=df["sale_price"]-df["cost_price"]  #profit
df.to_csv("orders.csv", index=False)
df.head(3)

In [ ]:
# Splitting data frame into 2 dataframe
df1 = df[['order_id', 'order_date', 'ship_mode','segment','country','city','state','postal_code','region','category','sub_category']]  # First half of columns
df2 = df[['order_id','product_id','cost_price','list_price','quantity','discount_percent','discount','sale_price','profit']]
df2.head(5)

In [30]:
import mysql.connector

connection = mysql.connector.connect(
  host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
  port = 4000,
  user = "2XWsHPpwQA4Z6ce.root",
  password = "lFu7DqRUZThvq3kn",
  database = "Retail_Project",
 
)
mycursor=connection.cursor()

In [ ]:
#creation of database
mycursor.execute("create database Retail_Project")

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# 1️⃣ MySQL/TiDB Connection Details
host = "gateway01.ap-southeast-1.prod.aws.tidbcloud.com",
port = 4000,
user = "2XWsHPpwQA4Z6ce.root",
password = "lFu7DqRUZThvq3kn",
database = "Retail_Project",

# 2️⃣ Create an SQLAlchemy Engine
engine = create_engine(f"mysql+mysqlconnector://{"2XWsHPpwQA4Z6ce.root"}:{"lFu7DqRUZThvq3kn"}@{"gateway01.ap-southeast-1.prod.aws.tidbcloud.com"}/{"Retail_Project"}")

# 3️⃣ Read CSV into DataFrame
df = pd.read_csv("orders.csv")

# 4️⃣ Move DataFrame into SQL Table
df.to_sql("orders_table", engine, if_exists="replace", index=False)

# AFTER SPLITING DATAFRAME MOVING INTO SQL TABLE 
df1.to_sql("ORDER1", engine, if_exists="replace", index=False)
df2.to_sql("ORDER2", engine, if_exists="replace", index=False)

In [ ]:
#Changing order id into primary key in order 1
mycursor.execute("ALTER TABLE Retail_Project.ORDER1 ADD PRIMARY KEY (order_id);")


In [45]:
#Changing order id into foreign key in order 2
mycursor.execute(
    """ALTER TABLE Retail_Project.ORDER2 
    ADD CONSTRAINT fk_order_id 
    FOREIGN KEY (order_id) 
    REFERENCES Retail_Project.ORDER1(order_id)""")

In [ ]:
# 18 Find the Profit percentage per Sub_Category
mycursor.execute(
        """SELECT df1.sub_category,SUM(df2.profit) AS Total_Profit
        FROM RETAIL_PROJECT.ORDER1 df1
        JOIN RETAIL_PROJECT.ORDER2 df2 ON df1.order_id = df2.order_id   
        GROUP BY df1.sub_category
        ORDER BY Total_Profit DESC
        """)
out=mycursor.fetchall()
from tabulate import tabulate
print(tabulate(out,headers=[i[0] for i in mycursor.description],  tablefmt='psql'))
